In [ ]:
%load_ext autoreload
%autoreload 2
from utils import dataloader, generate, plot_1d_statistic_over_time, plot_loss
from models import Forward, Posterior, Prior, Decoder
from train import run, val_step, DEVICE
import os, pickle, random, shutil, pandas as pd, torch
os.environ["KERAS_BACKEND"] = "torch"
import keras

# Split data
Make sure all data files are stored somewhere in `"./data/"`.

In [ ]:
files = [os.path.join(root, file) for root, _, files in os.walk("./data/") for file in files if file.endswith(".npz")]
random.seed(42)
random.shuffle(files)

train_files = files[:int(0.6 * len(files))]
val_files = files[int(0.6 * len(files)):int(0.8 * len(files))]
test_files = files[int(0.8 * len(files)):]

os.makedirs("./data/train", exist_ok=True)
os.makedirs("./data/val", exist_ok=True)
os.makedirs("./data/test", exist_ok=True)

for f in train_files:
    if not f.split("/")[-1] in os.listdir("./data/train"):
        shutil.move(f, "./data/train")
for f in val_files:
    if not f.split("/")[-1] in os.listdir("./data/val"):
        shutil.move(f, "./data/val")
for f in test_files:
    if not f.split("/")[-1] in os.listdir("./data/test"):
        shutil.move(f, "./data/test")

# Train

In [ ]:
# Instantiate models
forward_t = Forward()
forward_tplus1 = Forward()
prior = Prior()
posterior = Posterior()
decoder = Decoder()

# Instantiate optimizer
opt = keras.optimizers.AdamW(1e-4)

# Get data
train_loader = dataloader(data_dir="./data/train", batch_size=32)
val_loader = dataloader(data_dir="./data/val", batch_size=32)
test_loader = dataloader(data_dir="./data/test", batch_size=32)

#B Input parameter
b_field = pd.read_csv('b-field.csv', delimiter=',', index_col=0)
b_field_values = b_field.values.flatten()  # Convert to a 1D Numpy array
b_field = torch.tensor(b_field_values, dtype=torch.float32).to(DEVICE)
x = torch.linspace(0, 500, steps=500, requires_grad=True).to(DEVICE)

# Run training
save_dir = "./results/finitedifference"
run(train_loader, val_loader, forward_t, forward_tplus1, prior, posterior, decoder, opt, b_field, x, save_dir, 200, 10)

# # Plot loss
plot_loss(f"{save_dir}/history.json");

# Evaluate

In [ ]:
# Load trained models
forward_t.load_weights(f"{save_dir}/forward_t.weights.h5")
prior.load_weights(f"{save_dir}/prior.weights.h5")
decoder.load_weights(f"{save_dir}/decoder.weights.h5")

# Evaluate by calculating loss over test set
test_loss = 0
for i, (x_t, x_tplus1) in enumerate(test_loader):
    test_loss += val_step(x_t, x_tplus1, forward_t, prior, decoder)
test_loss /= i+1
print("Test reconstruction loss:", test_loss)

# Generate
Make sure the trained models have been loaded with the code cell above.

In [ ]:
# Do it for various starting points
for j in range(4):
    # Select one random ground truth starting point
    trajectory = test_loader.dataset.get_trajectory(pushforward=False)
    # Generate multiple trajectories from one starting point
    trajectory_hats = []
    for i in range(6):
        # Generate trajectory
        trajectory_hat = generate(trajectory, forward_t, prior, decoder)
        trajectory_hats.append(keras.ops.concatenate(trajectory_hat))
        # Save trajectories as figure
        fig = plot_1d_statistic_over_time(trajectory_hats[i].detach().cpu(), 0, "I don't know what this variable is");
        fig.savefig(f"{save_dir}/gen_{j}-{i}")
    # Save trajectories as tensors
    trajectory_hats = keras.ops.stack(trajectory_hats)
    with open(f"{save_dir}/generated_trajectories{j}.pkl", "wb") as file:
        pickle.dump(trajectory_hats, file)